In [1]:
# Train-Test Split 
import os
import shutil
import random

# BASE PATH
base_path = "dataset"


# Creating images and labels folder under dataset
images_dir = os.path.join(base_path, "images")
labels_dir = os.path.join(base_path, "labels")


# Creating train val test in images and labels folder 
splits = ["train", "val", "test"]
for split in splits:
    os.makedirs(os.path.join(base_path, "images", split), exist_ok=True)
    os.makedirs(os.path.join(base_path, "labels", split), exist_ok=True)


# Getting all the images 
image_files = [f for f in os.listdir(images_dir) if f.endswith((".jpg", ".png", ".jpeg"))]


# Shuffling and random splitting
random.shuffle(image_files)


# Split ratios
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1


total = len(image_files)
train_end = int(total * train_ratio)
val_end = train_end + int(total * val_ratio)


# Dividing the images in train val and test
train_files = image_files[:train_end]
val_files = image_files[train_end:val_end]
test_files = image_files[val_end:]


# Moving the images
def move_files(file_list, split_name):
    for img in file_list:
        # getting the image
        img_src = os.path.join(images_dir, img)
        # getting the label
        lbl_src = os.path.join(labels_dir, img.replace(".jpg", ".txt").replace(".png", ".txt").replace(".jpeg", ".txt")) 

        # deciding the destination to copy 
        img_dst = os.path.join(base_path, "images", split_name, img)
        lbl_dst = os.path.join(base_path, "labels", split_name, os.path.basename(lbl_src))

        # Move the image and label only if label exists
        if os.path.exists(lbl_src):
            shutil.move(img_src, img_dst)
            shutil.move(lbl_src, lbl_dst)

# func calls
move_files(train_files, "train")
move_files(val_files, "val")
move_files(test_files, "test")

print(f"Split Completed:")
print(f"Train: {len(train_files)} images")
print(f"Val:   {len(val_files)} images")
print(f"Test:  {len(test_files)} images")


Split Completed:
Train: 386 images
Val:   110 images
Test:  56 images


In [2]:
# YOLOv8 Training on GPU 
from ultralytics import YOLO
import torch

# Checking GPU
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

# 2️Loading YOLOv8 model
# Use yolov8n.pt for small, fast model
model = YOLO("yolov8n.pt")

# Training
model.train(
    data="data.yaml",      
    epochs=100,            
    imgsz=640,             # image size
    batch=4,               
    device=0,              # use GPU 0
    name="number_plate_exp",  
    save=True,             # save best.pt automatically
    augment=True,          # enable data augmentation
    verbose=True           # print epoch-wise info
)


CUDA available: True
GPU Name: NVIDIA GeForce RTX 3060 Laptop GPU
Ultralytics 8.3.228  Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=number_plate_exp2, nbs=64, nms=False, opset=None, optimize=False, optim

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002765E1FBF40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [8]:
from glob import glob
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Checking GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
if device == "cuda":
    print("GPU Name:", torch.cuda.get_device_name(0))

# Load trained YOLO model
model = YOLO("runs/detect/number_plate_exp2/weights/best.pt")

# Paths to test dataset
img_dir = "dataset/images/test"
label_dir = "dataset/labels/test"

# Getting all test images
image_paths = sorted(glob(os.path.join(img_dir, "*.jpg")) + glob(os.path.join(img_dir, "*.png")))

print(f"Found {len(image_paths)} test images\n")

y_true = []
y_pred = []

for i, img_path in enumerate(image_paths, 1):
    print(f"Processing image {i}/{len(image_paths)}: {os.path.basename(img_path)}")
    
    # Actual label 
    base = os.path.basename(img_path).rsplit(".", 1)[0]
    label_path = os.path.join(label_dir, base + ".txt")

    # checking the label
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            content = f.read().strip()
        if content == "":
            y_true.append(0)  # no_plate
        else:
            y_true.append(1)  # number_plate
    else:
        y_true.append(0)  # no_plate if label file missing

    # Prediction 
    result = model(img_path, device=device)[0]
    if len(result.boxes) > 0:
        y_pred.append(1)  # detected number plate
    else:
        y_pred.append(0)  # no detection

# Convert to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Metrics
print("\n=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=["no_plate", "number_plate"], zero_division=0))

print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_true, y_pred))

Using device: cuda
GPU Name: NVIDIA GeForce RTX 3060 Laptop GPU
Found 56 test images

Processing image 1/56: 20230614_150819ll.jpg

image 1/1 C:\Users\Anisha\Project_soft\dataset\images\test\20230614_150819ll.jpg: 640x480 1 number_plate, 23.7ms
Speed: 4.2ms preprocess, 23.7ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 480)
Processing image 2/56: 20230614_150952ll.jpg

image 1/1 C:\Users\Anisha\Project_soft\dataset\images\test\20230614_150952ll.jpg: 640x480 2 number_plates, 22.4ms
Speed: 4.4ms preprocess, 22.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)
Processing image 3/56: 20230620_160009ll.jpg

image 1/1 C:\Users\Anisha\Project_soft\dataset\images\test\20230620_160009ll.jpg: 640x480 2 number_plates, 22.4ms
Speed: 4.0ms preprocess, 22.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)
Processing image 4/56: 20230620_160023ll.jpg

image 1/1 C:\Users\Anisha\Project_soft\dataset\images\test\20230620_160023ll.jpg: 640x480 1 number

In [7]:
# YOLOv8 Detection + Crop & Save Number Plates

from PIL import Image  # to crop and save images

# Loading trained model
model = YOLO("runs/detect/number_plate_exp2/weights/best.pt")

# Paths
img_dir = "dataset/images/test"
save_dir = "cropped_plates"
os.makedirs(save_dir, exist_ok=True)

# Getting all test images
image_paths = sorted(glob(os.path.join(img_dir, "*.jpg")) + glob(os.path.join(img_dir, "*.png")) + glob(os.path.join(img_dir, "*.jpeg")))

# Run and crop plates
for img_path in image_paths:
    # Run YOLO 
    results = model(img_path, device=0)[0]

    # Open image for cropping
    img = Image.open(img_path)

    # Loop through detected boxes
    for i, box in enumerate(results.boxes.xyxy):  # xyxy = [x1, y1, x2, y2]
        x1, y1, x2, y2 = map(int, box)
        cropped_plate = img.crop((x1, y1, x2, y2))

        # Convert to RGB if image has alpha channel
        if cropped_plate.mode != "RGB":
            cropped_plate = cropped_plate.convert("RGB")

        # Save cropped image
        base_name = os.path.basename(img_path).split(".")[0]
        save_path = os.path.join(save_dir, f"{base_name}_plate{i+1}.jpg")
        cropped_plate.save(save_path)

        print(f"Cropped plate saved: {save_path}")

print("Cropping done for all detected plates!")



image 1/1 C:\Users\Anisha\Project_soft\dataset\images\test\20230614_150819ll.jpg: 640x480 1 number_plate, 24.6ms
Speed: 5.5ms preprocess, 24.6ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 480)
Cropped plate saved: cropped_plates\20230614_150819ll_plate1.jpg

image 1/1 C:\Users\Anisha\Project_soft\dataset\images\test\20230614_150952ll.jpg: 640x480 2 number_plates, 22.4ms
Speed: 4.2ms preprocess, 22.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)
Cropped plate saved: cropped_plates\20230614_150952ll_plate1.jpg
Cropped plate saved: cropped_plates\20230614_150952ll_plate2.jpg

image 1/1 C:\Users\Anisha\Project_soft\dataset\images\test\20230620_160009ll.jpg: 640x480 2 number_plates, 22.3ms
Speed: 4.2ms preprocess, 22.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 480)
Cropped plate saved: cropped_plates\20230620_160009ll_plate1.jpg
Cropped plate saved: cropped_plates\20230620_160009ll_plate2.jpg

image 1/1 C:\Users\Anisha\Project_soft\

In [7]:
# YOLOv8 Video Detection + Crop & Save Number Plates (skip 36 frames)

import cv2

# Paths config
video_path = "input_video.mp4"
save_dir = "cropped_plates_video"
os.makedirs(save_dir, exist_ok=True)

# Open video
cap = cv2.VideoCapture(video_path)
frame_count = 0
skip_frames = 36

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    # Skip frames
    if frame_count % skip_frames != 0:
        continue

    # Convert BGR (OpenCV) to RGB (PIL)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(frame_rgb)

    # Run YOLO 
    results = model(frame, device=0)[0]

    for i, box in enumerate(results.boxes.xyxy):
        x1, y1, x2, y2 = map(int, box)
        cropped_plate = img_pil.crop((x1, y1, x2, y2))

        # Save cropped plate
        save_path = os.path.join(save_dir, f"frame{frame_count}_plate{i+1}.jpg")
        cropped_plate.save(save_path)
        print(f"[Frame {frame_count}] Cropped plate saved: {save_path}")

cap.release()
print("Video processing done!")



0: 384x640 (no detections), 18.6ms
Speed: 83.9ms preprocess, 18.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 17.1ms
Speed: 3.5ms preprocess, 17.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 16.9ms
Speed: 3.1ms preprocess, 16.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 number_plates, 17.0ms
Speed: 3.1ms preprocess, 17.0ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)
[Frame 144] Cropped plate saved: cropped_plates_video\frame144_plate1.jpg
[Frame 144] Cropped plate saved: cropped_plates_video\frame144_plate2.jpg
[Frame 144] Cropped plate saved: cropped_plates_video\frame144_plate3.jpg

0: 384x640 5 number_plates, 17.2ms
Speed: 4.6ms preprocess, 17.2ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)
[Frame 180] Cropped plate saved: cropped_plates_video\frame180_plate1.jpg
[Frame 180] Cropped plate saved: cropp